## Mobilenet For Emotion Recognition  

In [0]:
!pip install tensorflow-gpu==2.0.0

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!rm -r ./train/
!rm -r ./valid/
!rm -r ./test/

In [0]:
!tar -xf "./drive/My Drive/Datasets/fer_6_classes.tar.xz"

In [0]:
!mkdir ravdess

In [0]:
!tar -C "./ravdess" -xf "./drive/My Drive/Datasets/ravdess.tar.xz"

In [0]:
# !rm ./train/neutral/fer??????[0-2]*
# !rm ./train/fear/fer??????[0-4]*
# !rm ./train/sadness/fer??????[0-4]*
# !rm ./train/happiness/fer??????[0-4]*
# !rm ./train/anger/fer??????[0-4]*
# !rm ./train/surprise/fer??????[0-4]*

In [0]:
print(len(os.listdir("./train/surprise")))
print(len(os.listdir("./train/neutral")))
print(len(os.listdir("./train/fear")))
print(len(os.listdir("./train/sadness")))
print(len(os.listdir("./train/happiness")))

5032
14289
8354
11981
15595


In [0]:
!cp ./ravdess/train/angry/*.* ./train/anger/
!cp ./ravdess/train/fear/*.* ./train/fear/
!cp ./ravdess/train/happy/*.* ./train/happiness/
!cp ./ravdess/train/neutral/*.* ./train/neutral/
!cp ./ravdess/train/sad/*.* ./train/sadness/
!cp ./ravdess/train/surprised/*.* ./train/surprise/

In [0]:
!cp ./ravdess/valid_test/angry/02-01* ./valid/anger/
!cp ./ravdess/valid_test/fear/02-01* ./valid/fear/
!cp ./ravdess/valid_test/happy/02-01* ./valid/happiness/
!cp ./ravdess/valid_test/neutral/02-01* ./valid/neutral/
!cp ./ravdess/valid_test/sad/02-01* ./valid/sadness/
!cp ./ravdess/valid_test/surprised/02-01* ./valid/surprise/

In [0]:
!cp ./ravdess/valid_test/angry/02-02* ./train/anger/
!cp ./ravdess/valid_test/fear/02-02* ./train/fear/
!cp ./ravdess/valid_test/happy/02-02* ./train/happiness/
!cp ./ravdess/valid_test/neutral/02-02* ./train/neutral/
!cp ./ravdess/valid_test/sad/02-02* ./train/sadness/
!cp ./ravdess/valid_test/surprised/02-02* ./train/surprise/

cp: cannot stat './ravdess/valid_test/surprised/02-02*': No such file or directory


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import csv

import tensorflow as tf
from tensorflow import keras
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.version.VERSION)

2.0.0


In [0]:
BATCH_SIZE = 128
IMG_HEIGHT = 96
IMG_WIDTH = 96
IMG_SIZE = 96
train_dir = "./train"
test_dir = "./test"
valid_dir = "./valid"
CLASS_NAMES = np.array(['anger', 'fear', 'happiness', 'neutral', 'sadness', 'surprise'])

In [0]:
data_generator = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_data_generator = ImageDataGenerator(rescale = 1./255.)

In [0]:
train_data_gen = data_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))
# ravdess_data_gen = data_generator.flow_from_directory(batch_size=BATCH_SIZE,
#                                                directory="./ravdess/train/",
#                                                shuffle=True,
#                                                target_size=(IMG_HEIGHT, IMG_WIDTH))
valid_data_gen = test_data_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=valid_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))
test_data_gen = test_data_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=test_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

Found 28026 images belonging to 6 classes.
Found 3497 images belonging to 6 classes.
Found 3497 images belonging to 6 classes.


In [0]:
model = tf.keras.models.load_model("./drive/My Drive/Models/mobilenetv2")

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_96 (Model)  multiple                  2257984   
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  7686      
Total params: 2,265,670
Trainable params: 2,231,558
Non-trainable params: 34,112
_________________________________________________________________


In [0]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
mobilenet = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [0]:
small_mobilenet = tf.keras.models.Model(inputs=mobilenet.layers[0].input, outputs=mobilenet.layers[-4].output) # -4-9
small_mobilenet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 97, 97, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 48, 48, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 48, 48, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

In [0]:
regularizer = tf.keras.regularizers.l2()
model = tf.keras.Sequential([
  small_mobilenet,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dropout(0.3),
  keras.layers.Dense(64, activation="relu"),
  keras.layers.Dropout(0.3),
  keras.layers.Dense(2, activation="tanh"),
  keras.layers.Dense(6, activation="softmax")
])

In [0]:
num_train = 28026
num_test = 3497
num_valid = 3497
steps_per_epoch = round(num_train)//BATCH_SIZE
validation_steps = round(num_valid)//BATCH_SIZE

In [0]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=steps_per_epoch*1000,
  decay_rate=1,
  staircase=False)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [0]:
model.evaluate(valid_data_gen)
model.evaluate(test_data_gen)

28/28 [==============================] - 2s 64ms/step - loss: 2.0495 - accuracy: 0.1172


[2.0495401535715376, 0.11724335]

In [0]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

history = model.fit_generator(train_data_gen,
                    callbacks=[callback],
                    steps_per_epoch=steps_per_epoch,
                    epochs=100,
                    validation_data=valid_data_gen,
                    validation_steps=validation_steps)
model.save("./drive/My Drive/Models/mobilenetv2_small_bottleneck")
model.evaluate(test_data_gen)

Epoch 1/100
218/218 [==============================] - 113s 518ms/step - loss: 1.4901 - accuracy: 0.4581 - val_loss: 1.4895 - val_accuracy: 0.4679
Epoch 2/100
218/218 [==============================] - 110s 506ms/step - loss: 1.4189 - accuracy: 0.4977 - val_loss: 1.6982 - val_accuracy: 0.3154
Epoch 3/100
218/218 [==============================] - 110s 506ms/step - loss: 1.3252 - accuracy: 0.5390 - val_loss: 1.9600 - val_accuracy: 0.2659
Epoch 4/100
218/218 [==============================] - 111s 508ms/step - loss: 1.2018 - accuracy: 0.6004 - val_loss: 2.4922 - val_accuracy: 0.1603
Epoch 5/100
218/218 [==============================] - 110s 504ms/step - loss: 1.0972 - accuracy: 0.6437 - val_loss: 1.3112 - val_accuracy: 0.6047
Epoch 6/100
218/218 [==============================] - 109s 501ms/step - loss: 1.0220 - accuracy: 0.6582 - val_loss: 1.5590 - val_accuracy: 0.5486
Epoch 7/100
218/218 [==============================] - 110s 505ms/step - loss: 0.9652 - accuracy: 0.6761 - val_loss: 1